<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Gather-Follower-&amp;-Statuses-Counts-for-Podcasts-with-Twitter-Accounts" data-toc-modified-id="Gather-Follower-&amp;-Statuses-Counts-for-Podcasts-with-Twitter-Accounts-1">Gather Follower &amp; Statuses Counts for Podcasts with Twitter Accounts</a></span><ul class="toc-item"><li><span><a href="#Goal-is-to-use-these-to-enrich-our-understanding-of-podcast-social-activity." data-toc-modified-id="Goal-is-to-use-these-to-enrich-our-understanding-of-podcast-social-activity.-1.1">Goal is to use these to enrich our understanding of podcast social activity.</a></span></li></ul></li></ul></div>

In [31]:
import pickle

import features

import time
import re

import twitter
api = twitter.Api(consumer_key='3ExAhrEEf6c1zhtFzswFJ7cGD',
  consumer_secret='tnyOfCRVq4YT6uCo6IVChmQHCX3iRTK1Yg45ZqWqAW2EjUQwKO',
  access_token_key='1166032070667202565-9MbVIOwbCC8wnVarMsIE2NkBDwADix',
  access_token_secret='OpCglzB7WsTyeEjI7CeNpo1cHTJSYQqKe4ylOrQpSBlSx')

In [32]:
print(api.VerifyCredentials())

{"created_at": "Mon Aug 26 16:59:13 +0000 2019", "default_profile": true, "description": "data scientist | meditator | Coloradan in BK/NYC", "favourites_count": 1, "followers_count": 4, "friends_count": 20, "id": 1166032070667202565, "id_str": "1166032070667202565", "name": "Thomas Kavanagh", "profile_background_color": "F5F8FA", "profile_image_url": "http://pbs.twimg.com/profile_images/1166032951932403714/lkQcnMwS_normal.jpg", "profile_image_url_https": "https://pbs.twimg.com/profile_images/1166032951932403714/lkQcnMwS_normal.jpg", "profile_link_color": "1DA1F2", "profile_sidebar_border_color": "C0DEED", "profile_sidebar_fill_color": "DDEEF6", "profile_text_color": "333333", "profile_use_background_image": true, "screen_name": "ThomasK26635081"}


## Gather Follower & Statuses Counts for Podcasts with Twitter Accounts

### Goal is to use these to enrich our understanding of podcast social activity.

Tutorial found here:

https://www.alexkras.com/how-to-get-user-feed-with-twitter-api-and-python/

We will unload (any) version of the sanitized dataframe, in order to gather a list of channels in order to create a dictionary.

In [3]:
df_version = 'merged_10-06_10-17.pickle'
import_path = '../scraped/merged/' + df_version
with open(import_path, 'rb') as file:
    raw_df = pickle.load(file)

raw_df['twitter'] = raw_df['ch_feed-socials'].apply(lambda row: features.has_domain(row, 'twitter'))

In [4]:
t_cols = ['title','chan_url','ch_feed-socials','twitter']

In [5]:
raw_df.sample(3)[t_cols]

,title,chan_url,ch_feed-socials,twitter
21,WTF with Marc Maron Podcast,https://castbox.fm/channel/WTF-with-Marc-Maron...,"[https://twitter.com/WTFpod, http://wtfpod.com]",1
81,Oh Boy by Man Repeller,https://castbox.fm/channel/Oh-Boy-by-Man-Repel...,[http://www.manrepeller.com/category/oh-boy],0
39,What We're Tasting,https://castbox.fm/channel/What-We're-Tasting-...,[http://www.winemag.com/wwt],0


In [6]:
raw_df[raw_df.twitter == 1].sample(3)[t_cols]

,title,chan_url,ch_feed-socials,twitter
154,SmoshCast,https://castbox.fm/channel/SmoshCast-id2026252,"[https://twitter.com/smosh, https://www.facebo...",1
26,Alt.Latino,https://castbox.fm/channel/Alt.Latino-id432033,"[https://twitter.com/NPR, https://www.facebook...",1
51,TANIS,https://castbox.fm/channel/TANIS-id167,"[https://twitter.com/tanispodcast, https://www...",1


In [ ]:
raw_df.to_dict(orient='records')

In [8]:
twitter_chans = raw_df[raw_df.twitter == 1][['title','chan_url','ch_feed-socials']].to_dict(orient='records')

In [23]:
def extract_twitter_screen_name(ch_feed_socials):

    if len(ch_feed_socials) == 0:
        return ''
    else:
        pattern = '.+(twitter.com/)(?P<screen_name>.+)'
        for url in ch_feed_socials:
#             print(url)
            try:
                screen_name = re.match(pattern, url).group('screen_name')
                if screen_name:
                    return screen_name
            except:
                oass
        return ''

In [93]:
for chan in twitter_dict.keys():
    twitter_dict[chan]['screen_name'] = extract_twitter_screen_name(twitter_dict[chan]['ch_feed-socials'])

In [96]:
with open('../social_metrics/twitter/channel_stats_by_name.pickle', '+') as file:
    pickle.dump(twitter_dict, file)

In [10]:
twitter_dict = {}
twitter_dict

{}

Let's create a set of unique twitter handles, as its own dataframe, which we will use as a reference.

In [11]:
# Twitter api usage example
npr = api.GetUser(screen_name='nprfreshair')
npr.followers_count

276441

In [15]:
npr.statuses_count

25857

In [100]:
with open('../social_metrics/twitter/channel_stats_by_name.pickle', 'rb') as file:
    twitter_follower_dict = pickle.load(file)

In [12]:
def add_channels_to_twitter_dictionary(channnels, output_dict):
    for chan in channnels:
        key = chan['title']
        try:
            output_dict[key].update(chan)
        except:
            output_dict[key] = chan
    return True

In [13]:
twitter_dict

{}

In [15]:
twitter_chans[:5]

[{'title': 'Small Town Horror',
  'chan_url': 'https://castbox.fm/channel/Small-Town-Horror-id1266396',
  'ch_feed-socials': ['https://twitter.com/SmallTwnHorror',
   'https://www.facebook.com/smalltownhorrorpodcast',
   'https://play.acast.com/s/smalltownhorror']},
 {'title': 'I Should Be Writing',
  'chan_url': 'https://castbox.fm/channel/I-Should-Be-Writing-id210',
  'ch_feed-socials': ['https://twitter.com/mightymur',
   'http://murverse.com/tag/isbw/']},
 {'title': 'Book Riot - The Podcast',
  'chan_url': 'https://castbox.fm/channel/Book-Riot-The-Podcast-id1207097',
  'ch_feed-socials': ['https://twitter.com/BookRiot',
   'https://bookriot.com/listen/shows/thepodcast/']},
 {'title': 'The Disney Story Origins Podcast',
  'chan_url': 'https://castbox.fm/channel/The-Disney-Story-Origins-Podcast-id7837',
  'ch_feed-socials': ['https://twitter.com/pauljhaleauthor',
   'http://www.disneystoryoriginspodcast.com/category/podcast/']},
 {'title': 'Saga',
  'chan_url': 'https://castbox.fm/ch

In [16]:
add_channels_to_twitter_dictionary(twitter_chans, twitter_dict)

True

In [19]:
[(k, twitter_dict[k]) for k in list(twitter_dict.keys())[:5]]

[('Small Town Horror',
  {'title': 'Small Town Horror',
   'chan_url': 'https://castbox.fm/channel/Small-Town-Horror-id1266396',
   'ch_feed-socials': ['https://twitter.com/SmallTwnHorror',
    'https://www.facebook.com/smalltownhorrorpodcast',
    'https://play.acast.com/s/smalltownhorror']}),
 ('I Should Be Writing',
  {'title': 'I Should Be Writing',
   'chan_url': 'https://castbox.fm/channel/I-Should-Be-Writing-id210',
   'ch_feed-socials': ['https://twitter.com/mightymur',
    'http://murverse.com/tag/isbw/']}),
 ('Book Riot - The Podcast',
  {'title': 'Book Riot - The Podcast',
   'chan_url': 'https://castbox.fm/channel/Book-Riot-The-Podcast-id1207097',
   'ch_feed-socials': ['https://twitter.com/BookRiot',
    'https://bookriot.com/listen/shows/thepodcast/']}),
 ('The Disney Story Origins Podcast',
  {'title': 'The Disney Story Origins Podcast',
   'chan_url': 'https://castbox.fm/channel/The-Disney-Story-Origins-Podcast-id7837',
   'ch_feed-socials': ['https://twitter.com/pauljh

In [21]:
def update_screen_names(t_dict):
    
    chan_list = t_dict.keys()
    
    for chan in chan_list:
        if 'screen_name' in t_dict[chan].keys():
            pass
        else:
            socials = t_dict[chan]['ch_feed-socials']
            t_dict[chan]['screen_name'] = extract_twitter_screen_name(socials)
    
    return True

In [24]:
update_screen_names(twitter_dict)

True

In [25]:
[(k, twitter_dict[k]) for k in list(twitter_dict.keys())[:5]]

[('Small Town Horror',
  {'title': 'Small Town Horror',
   'chan_url': 'https://castbox.fm/channel/Small-Town-Horror-id1266396',
   'ch_feed-socials': ['https://twitter.com/SmallTwnHorror',
    'https://www.facebook.com/smalltownhorrorpodcast',
    'https://play.acast.com/s/smalltownhorror'],
   'screen_name': 'SmallTwnHorror'}),
 ('I Should Be Writing',
  {'title': 'I Should Be Writing',
   'chan_url': 'https://castbox.fm/channel/I-Should-Be-Writing-id210',
   'ch_feed-socials': ['https://twitter.com/mightymur',
    'http://murverse.com/tag/isbw/'],
   'screen_name': 'mightymur'}),
 ('Book Riot - The Podcast',
  {'title': 'Book Riot - The Podcast',
   'chan_url': 'https://castbox.fm/channel/Book-Riot-The-Podcast-id1207097',
   'ch_feed-socials': ['https://twitter.com/BookRiot',
    'https://bookriot.com/listen/shows/thepodcast/'],
   'screen_name': 'BookRiot'}),
 ('The Disney Story Origins Podcast',
  {'title': 'The Disney Story Origins Podcast',
   'chan_url': 'https://castbox.fm/cha

In [51]:
def get_follower_counts(t_dict, api_calls=1):
    
    chan_list = t_dict.keys()
    
    for chan in chan_list:
        
#         time.sleep(3)

        if api_calls < 1:
            return 'local API limit reached'
        if 'screen_name' in t_dict[chan].keys():
            sn = t_dict[chan]['screen_name']
#             print(f'about to scan {chan} with sn {sn}')
            if 'follower_count' in t_dict[chan].keys():
#                 print('already calculated follower count for ',chan)
                continue
            else:
                try:
                    user = api.GetUser(screen_name=sn)
                    follower_count = user.followers_count

                    print(f'user {sn} has {follower_count} followers')
                    t_dict[chan]['follower_count'] = follower_count

                    api_calls = api_calls - 1

                except:
                    t_dict[chan]['follower_count'] = 0
                    print(f'failed to pull follower count for {chan}')
                    continue
        else:
            continue

In [52]:
get_follower_counts(twitter_dict, api_calls=4000)

In [53]:
twitter_dict

{'Small Town Horror': {'title': 'Small Town Horror',
  'chan_url': 'https://castbox.fm/channel/Small-Town-Horror-id1266396',
  'ch_feed-socials': ['https://twitter.com/SmallTwnHorror',
   'https://www.facebook.com/smalltownhorrorpodcast',
   'https://play.acast.com/s/smalltownhorror'],
  'screen_name': 'SmallTwnHorror',
  'follower_count': 0},
 'I Should Be Writing': {'title': 'I Should Be Writing',
  'chan_url': 'https://castbox.fm/channel/I-Should-Be-Writing-id210',
  'ch_feed-socials': ['https://twitter.com/mightymur',
   'http://murverse.com/tag/isbw/'],
  'screen_name': 'mightymur',
  'follower_count': 14970},
 'Book Riot - The Podcast': {'title': 'Book Riot - The Podcast',
  'chan_url': 'https://castbox.fm/channel/Book-Riot-The-Podcast-id1207097',
  'ch_feed-socials': ['https://twitter.com/BookRiot',
   'https://bookriot.com/listen/shows/thepodcast/'],
  'screen_name': 'BookRiot',
  'follower_count': 214860},
 'The Disney Story Origins Podcast': {'title': 'The Disney Story Origin

In [54]:
with open('../social_metrics/twitter/channel_stats_by_name_oct6_11p.pickle', 'wb') as file:
    pickle.dump(twitter_dict, file)